In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import numpy as np
import torch.nn.functional as F
import mlflow

In [2]:
!gdown --id 1R5JOvOnBo5dGsbqM7fcs_QNcQ3Ht4E1a

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1R5JOvOnBo5dGsbqM7fcs_QNcQ3Ht4E1a
To: /content/archive_books.zip
100% 26.5M/26.5M [00:00<00:00, 124MB/s]


In [3]:
!unzip ./archive_books.zip

Archive:  ./archive_books.zip
  inflating: 11-0.txt                
  inflating: __MACOSX/._11-0.txt     
  inflating: 16-0.txt                
  inflating: __MACOSX/._16-0.txt     
  inflating: 23-0.txt                
  inflating: __MACOSX/._23-0.txt     
  inflating: 35-0.txt                
  inflating: __MACOSX/._35-0.txt     
  inflating: 41-0.txt                
  inflating: __MACOSX/._41-0.txt     
  inflating: 43-0.txt                
  inflating: __MACOSX/._43-0.txt     
  inflating: 55-0.txt                
  inflating: __MACOSX/._55-0.txt     
  inflating: 74-0.txt                
  inflating: __MACOSX/._74-0.txt     
  inflating: 76-0.txt                
  inflating: __MACOSX/._76-0.txt     
  inflating: 84-0.txt                
  inflating: __MACOSX/._84-0.txt     
  inflating: 98-0.txt                
  inflating: __MACOSX/._98-0.txt     
  inflating: 103-0.txt               
  inflating: __MACOSX/._103-0.txt    
  inflating: 113-0.txt               
  inflating: __MACOS

In [4]:
books = []
for filename in os.listdir():
   if filename.endswith('.txt'):
    with open(filename, 'r') as f:
        text = f.read()
        books.append(text)

In [5]:
len(books)

99

In [6]:
processed_data = ''
for element in books:
  try:
    processed_data += element
  except:
    pass

In [7]:
len(processed_data)

68795566

In [8]:
#Tokenization
chars = set(processed_data)
int2char = {idx: char for idx, char in enumerate(chars)}
char2int = {char: idx for idx, char in enumerate(chars)}

In [9]:
encoded = np.array([char2int[ch] for ch in processed_data])

In [3]:
#One-hot encoding
def one_hot_encode(arr, n_labels):
    
    one_hot = np.eye(n_labels)[arr].astype(np.float32)
    
    return one_hot

In [11]:
#Mini-batches
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = len(arr) // batch_size_total
    
    ## Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    
    ## Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        # The target is a version of x shifted by one (do not forget border conditions)
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [12]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [13]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)

        # Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        # Define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        out, hidden = self.lstm(x, hidden)

        # Pass through a dropout layer
        out = self.dropout(out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        # Put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(DEVICE),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(DEVICE))

        
        return hidden

In [15]:
#Train the model 
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    net.to(DEVICE)
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    
                    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data

                checkpoint = {
                        'epoch': e + 1,
                        'state_dict': net.state_dict(),
                        'optimizer': opt.state_dict()
                    }
                torch.save(checkpoint, 'last_checkpoint.pt')

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [16]:
n_hidden = 512 
n_layers = 2  
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(298, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=298, bias=True)
)


In [18]:
batch_size = 64 
seq_length = 512  
n_epochs = 1  

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.01, print_every=10)

Epoch: 1/1... Step: 10... Loss: 3.2460... Val Loss: 3.1956
Epoch: 1/1... Step: 20... Loss: 3.2163... Val Loss: 3.1796
Epoch: 1/1... Step: 30... Loss: 3.2118... Val Loss: 3.1583
Epoch: 1/1... Step: 40... Loss: 3.1770... Val Loss: 3.1565
Epoch: 1/1... Step: 50... Loss: 3.1926... Val Loss: 3.1548
Epoch: 1/1... Step: 60... Loss: 3.1633... Val Loss: 3.1570
Epoch: 1/1... Step: 70... Loss: 3.2050... Val Loss: 3.1550
Epoch: 1/1... Step: 80... Loss: 3.1941... Val Loss: 3.1545
Epoch: 1/1... Step: 90... Loss: 3.1763... Val Loss: 3.1541
Epoch: 1/1... Step: 100... Loss: 3.1783... Val Loss: 3.1553
Epoch: 1/1... Step: 110... Loss: 3.1551... Val Loss: 3.1526
Epoch: 1/1... Step: 120... Loss: 3.1511... Val Loss: 3.1544
Epoch: 1/1... Step: 130... Loss: 3.1961... Val Loss: 3.1549
Epoch: 1/1... Step: 140... Loss: 3.1800... Val Loss: 3.1557
Epoch: 1/1... Step: 150... Loss: 3.1895... Val Loss: 3.1559
Epoch: 1/1... Step: 160... Loss: 3.1603... Val Loss: 3.1601
Epoch: 1/1... Step: 170... Loss: 3.2026... Val Lo

In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        

        inputs = inputs.to(DEVICE)
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [20]:
def sample(net, size, prime='The', top_k=None):
        
    net.to(DEVICE)
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [21]:
print(sample(net, 200, prime='son', top_k=5))

son to short, with the said
at the bonges as so makit the say at the pass, think in sore of my see said of at him a say of a cending, that
who were the breas any hight the still of the chints. And the
war


In [23]:
torch.save(net, 'net_v1.model')

In [38]:
model = torch.load('net_v1.model')

In [39]:
#continue training
batch_size = 64 
seq_length = 512  
n_epochs = 2  

# train the model
train(model, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.01, print_every=10)

Epoch: 1/2... Step: 10... Loss: 2.0668... Val Loss: 1.8671
Epoch: 1/2... Step: 20... Loss: 2.0279... Val Loss: 1.8395
Epoch: 1/2... Step: 30... Loss: 2.0066... Val Loss: 1.8279
Epoch: 1/2... Step: 40... Loss: 1.9974... Val Loss: 1.8164
Epoch: 1/2... Step: 50... Loss: 1.9733... Val Loss: 1.8093
Epoch: 1/2... Step: 60... Loss: 1.9539... Val Loss: 1.8030
Epoch: 1/2... Step: 70... Loss: 1.9630... Val Loss: 1.7990
Epoch: 1/2... Step: 80... Loss: 1.9684... Val Loss: 1.7914
Epoch: 1/2... Step: 90... Loss: 1.9453... Val Loss: 1.7833
Epoch: 1/2... Step: 100... Loss: 1.9428... Val Loss: 1.7733
Epoch: 1/2... Step: 110... Loss: 1.9125... Val Loss: 1.7617
Epoch: 1/2... Step: 120... Loss: 1.9070... Val Loss: 1.7579
Epoch: 1/2... Step: 130... Loss: 1.9073... Val Loss: 1.7503
Epoch: 1/2... Step: 140... Loss: 1.8930... Val Loss: 1.7438
Epoch: 1/2... Step: 150... Loss: 1.8821... Val Loss: 1.7351
Epoch: 1/2... Step: 160... Loss: 1.8601... Val Loss: 1.7259
Epoch: 1/2... Step: 170... Loss: 1.8758... Val Lo

In [40]:
print(sample(model, 200, prime='son', top_k=5))

sonde time at their shows. Wherefore is now saying he wouldn’t also serve the police of the case of all as is shall thou
are serving, war also with the will, the child in an
insublege and house of his bea


In [43]:
torch.save(model, 'net_v2.model')

In [44]:
import pickle
with open("encoded.pickle", "wb") as output_file:
  pickle.dump(encoded, output_file)

In [45]:
with open("chars.pickle", "wb") as output_file:
  pickle.dump(chars, output_file)

In [46]:
with open("int2char.pickle", "wb") as output_file:
  pickle.dump(int2char, output_file)

In [47]:
with open("char2int.pickle", "wb") as output_file:
  pickle.dump(char2int, output_file)

In [6]:
model = torch.load('net_v2.model', map_location=torch.device('cpu'))

In [ ]:
params = {
    'batch_size': 64,
    'seq_lenght': 512,
    'learning_rate': 0.01
}

metrics = {
    'train_loss': 1.5018,
    'val_loss': 1.3823
}

In [ ]:
# connect to server
MLFLOW_SERVER_URL = 'http://mlflow:5003/'
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

In [ ]:
experiment_name = 'char_rnn_books'
mlflow.set_experiment(experiment_name)

mlflow.end_run()
with mlflow.start_run():
    mlflow.log_param('batch_size', params['batch_size'])
    mlflow.log_param('seq_lenght', params['seq_lenght'])
    mlflow.log_param('learning_rate', params['learning_rate'])
    mlflow.log_metric('train_loss', metrics['train_loss'])
    mlflow.log_metric('val_loss', metrics['val_loss'])
    mlflow.pytorch.log_model(model, "model")